In [1]:
!pip install langchain-openai langchain-core tiktoken langchain langchain-community chainlit uvicorn pymupdf qdrant-client ragas

In [3]:
from langchain_openai import ChatOpenAI
import tiktoken
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from ragas.testset.generator import TestsetGenerator
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough
from utils import *
import os
import getpass
from langchain.globals import set_debug
from langchain_openai import ChatOpenAI, OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [5]:
openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
docs = PyMuPDFLoader("meta-10k.pdf").load()
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4-turbo")
test_generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    OpenAIEmbeddings()
)


In [10]:
def tiktoken_len(text) -> int:
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

def split_into_chunks():
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000, chunk_overlap=200, length_function=tiktoken_len
    )
    split_chunks = text_splitter.split_documents(docs)
    
    return split_chunks

def init_prompt() -> ChatPromptTemplate:
    RAG_PROMPT = """
        ###Instruction###:
        Answer the question based only on the following context. If you cannot answer the question with the context, please respond with "I don't know":
        
        CONTEXT:
        {context}

        QUERY:
        {question}            
        """
    rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)
    return rag_prompt

def get_vector_store(split_chunks):
    qdrant_vectorstore = Qdrant.from_documents(
        split_chunks,
        embedding_model,
        location=":memory:",
        collection_name="meta-10k",
    )
    return qdrant_vectorstore

def generate_test_set()-> None:
    text_splitter_eval = RecursiveCharacterTextSplitter(
        chunk_size = 600,
        chunk_overlap = 50
    )
    eval_documents = text_splitter_eval.split_documents(docs)
    distributions = {
        "simple": 0.5,
        "multi_context": 0.4,
        "reasoning": 0.1
    }
    testset = test_generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
    print(len(testset.to_pandas()))



In [11]:
rag_prompt_template = init_prompt()
split_chunks = split_into_chunks()

# normal retriever
qdrant_retriever = get_vector_store(split_chunks).as_retriever()

#Contexttual Compression
compressor = LLMChainExtractor.from_llm(OpenAI(temperature=0))
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=qdrant_retriever
)

# create a chain
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | openai_chat_model, "context": itemgetter("context")}
)

response = retrieval_augmented_qa_chain.invoke({"question" : "what is the value of Total cash and cash equivalents ?"})
print("response :"+ response["response"].content)



2024-05-02 10:56:27 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-02 10:56:29 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-02 10:56:31 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-02 10:56:31 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-02 10:56:32 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


2024-05-02 10:56:33 - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


2024-05-02 10:56:35 - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


2024-05-02 10:56:35 - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


2024-05-02 10:56:35 - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-05-02 10:56:36 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
response :$14,681
